In [1]:
import argparse
import copy
import glob
import json
import numpy as np
from molSimplifyAD.dbclass_mongo import tmcMongo
from molSimplifyAD.utils.pymongo_tools import connect2db, insert, count_find, convert2dataframe
from molSimplifyAD.ga_tools import get_mulliken

Using TensorFlow backend.


## Connect to DB

In [3]:
from os.path import expanduser
home = expanduser("~")
dbconfig = json.load(open(home + "/.db_config"))

db = connect2db(user=dbconfig['user'], pwd=dbconfig['pwd'],
                host="localhost", port=27017,
                database='tmc', auth=True)

In [5]:
db.oct.count() # Test your connection

/home/crduan/.conda/envs/mols_keras/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


163544

## Query db for documents needed for update

In [ ]:
### This is just an example. Change the constraints based on what you want.
constraints = {"alpha": {"$in": [20.0]},
               "lig6": {"$ne": "x"},
               "tag": "OxoHAT",
               "converged": True
               }
cursor = db.oct.find(constraints,
                     no_cursor_timeout=True).batch_size(10)

tot = count_find(cursor)
print(tot)
cursor = db.oct.find(constraints,
                     no_cursor_timeout=True).batch_size(10)

## Update document (This **ONLY** works for cases you do not need information from the DFTrun class object)

In [ ]:
failed_list = []
update_fields = ["ss_flag"] ## Only add keys you want to overwrite here.
count = 0
converted = 0
for _tmcdoc in cursor: ## This _tmcdoc is a db document (dictionary)
    tmcdoc_new = _tmcdoc.copy()
    ###----Your code here----
    ### Do whatever operation on the document tmcdoc_new
    ### But please be CAREFUL about what you are doing.
    ###----------------------
    ## Sanity check for the modification.
    for k in tmcdoc_new:
        if k in _tmcdoc and not k in update_fields:
            if not str(_tmcdoc[k]) == str(tmcdoc_new[k]):
                raise ValueError("Error on %s. If you want to overwrite existing key-value pairs in the document, add it to the <update_fields>"%k)
    for k in _tmcdoc:
        if not k in tmcdoc_new:
            raise ValueError("Error on %s. You are removing an existing key-value pair in the document. Please think twice before you do this."%k)
    print("replacing: ", _tmcdoc['_id'], _tmcdoc['unique_name'])
    db.oct.find_one_and_replace({"_id": _tmcdoc['_id']}, tmcdoc_new)
    count += 1
    if count % 100 == 0:
        print("processed %d, converted %d, total %d" % (count, converted, tot))